In [5]:
import pandas as pd
import numpy as np
import pickle

In [6]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

# Import Dataframe

In [7]:
# Load pickled dataframe

file = 'pickles/features_04_30.pkl'

with open(file, 'rb') as f:
    df_features = pickle.load(f)

In [8]:
df_features.head()

,"(., '')","(PRP, VBD)",``,"(,, '')","('', PRP, VBD)","(,, ADP)","(NOUN, VBD)",VBD,"(VBD, .)","(VBD, ,)",...,"('', PRP)","(,, VBD)",source,readability_SMOG,avg_sent_len,Num_Sentences,sd_sent_len,norm_stop_freq,norm_punct_freq,norm_funct_freq
0,0.0,0.000000,0.000000,0.0,0.0,0.013333,0.013333,0.013158,0.0,0.0,...,0.0,0.0,economist,11.2,20.666667,3,11.841546,0.421875,0.234375,0.390625
1,0.0,0.000000,0.000000,0.0,0.0,0.013514,0.000000,0.013333,0.0,0.0,...,0.0,0.0,economist,10.7,17.000000,4,3.391165,0.500000,0.132353,0.470588
2,0.0,0.000000,0.000000,0.0,0.0,0.012346,0.000000,0.000000,0.0,0.0,...,0.0,0.0,economist,5.7,13.800000,5,5.844656,0.422535,0.197183,0.436620
3,0.0,0.009346,0.000000,0.0,0.0,0.000000,0.000000,0.018519,0.0,0.0,...,0.0,0.0,economist,11.6,17.800000,5,7.626270,0.422222,0.255556,0.411111
4,0.0,0.000000,0.013889,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,economist,13.6,18.666667,3,7.408704,0.362069,0.327586,0.344828


# A bunch of classifiers

In [9]:
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,f1_score,precision_score,recall_score
from xgboost import XGBClassifier

/Users/sally/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
# prepare models
models = []
models.append(('LR', LogisticRegression(multi_class='multinomial',solver='lbfgs')))
#models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('XGBoost',XGBClassifier( n_estimators=1000)))
#models.append(('SVM', SVC(kernel = 'linear')))

In [12]:
# X -> features, y -> label
X=df_features.loc[:, df_features.columns != 'source']

#X=df_features[['readability_SMOG','avg_sent_len','sd_sent_len','norm_stop_freq','norm_punct_freq','norm_funct_freq']]
y=df_features.source
print(len(X))

52812


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.3, random_state = 0)

val_accuracy_all={}
val_fscore_all={}
val_precision_all={}
val_recall_all={}

train_accuracy_all={}
train_fscore_all={}
train_precision_all={}
train_recall_all={}

for name,model in models:
    
    print('Training ', name)
    clf=model.fit(X_train,y_train)
    
    print('Getting testing error ...')
    y_pred=clf.predict(X_test)
    
    val_precision_all[name]=precision_score(y_test,y_pred,average='macro')
    val_recall_all[name]=recall_score(y_test,y_pred,average='macro')
    val_fscore_all[name]=f1_score(y_test, y_pred, average='macro')
    val_accuracy_all[name]=accuracy_score(y_test, y_pred)

    print('Getting training error ...')
    y_train_pred=clf.predict(X_train)
    
    train_precision_all[name]=precision_score(y_train,y_train_pred,average='macro')
    train_recall_all[name]=recall_score(y_train,y_train_pred,average='macro')
    train_fscore_all[name]=f1_score(y_train,y_train_pred, average='macro')
    train_accuracy_all[name]=accuracy_score(y_train,y_train_pred)
    
    print('\n')

val_classifier_results=pd.DataFrame({'Accuracy':pd.Series(val_accuracy_all),'Precision':pd.Series(val_precision_all),'Recall':pd.Series(val_recall_all),'F1_Score':pd.Series(val_fscore_all)}) 

train_classifier_results=pd.DataFrame({'Accuracy':pd.Series(train_accuracy_all),'Precision':pd.Series(train_precision_all),'Recall':pd.Series(train_recall_all),'F1_Score':pd.Series(train_fscore_all)})  



Training  LR
Getting testing error ...
Getting training error ...


Training  KNN
Getting testing error ...
Getting training error ...


Training  NB
Getting testing error ...
Getting training error ...


Training  XGBoost
Getting testing error ...
Getting training error ...




In [14]:
print(train_classifier_results)

         Accuracy  F1_Score  Precision    Recall
KNN      0.573171  0.566496   0.574451  0.578462
LR       0.513255  0.497557   0.500262  0.501655
NB       0.427586  0.410950   0.494164  0.448846
XGBoost  0.693681  0.687154   0.685473  0.690629


In [15]:
print(val_classifier_results)

         Accuracy  F1_Score  Precision    Recall
KNN      0.382984  0.374119   0.373405  0.385623
LR       0.513065  0.495733   0.497527  0.500285
NB       0.429500  0.412919   0.492705  0.450919
XGBoost  0.629702  0.621602   0.619203  0.625379


# SVM

In [ ]:
# Grid search:

# vary kernels
# vary 'C'

# Record testing error AND training error

In [ ]:
#SVM with grid search. Needs to be completed

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['accuracy','precision_macro', 'recall_macro','f1_macro']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s' % score)
    clf.fit(X_train, y_train)
    
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)



# Logistic Regression

In [ ]:
# Try different regularization techniques

# If it's not overfitting, maybe try adding in some interaction terms?

# XGBOOST

In [35]:
params = {'min_child_weight': [3, 5, 7],
        'gamma': [0.25, 0.5, 0.75],
        'subsample': [0.4, 0.6, 0.8],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [1, 3, 5, 7],
        'learning_rate': [0.01, 0.025, 0.05, 0.1],
        }

skf = StratifiedKFold(n_splits=2, shuffle = True, random_state = 1001)

xgb = XGBClassifier(n_estimators=300, 
                    objective= 'multi:softmax', 
                    seed=27)

gsearch1 = GridSearchCV(estimator = xgb,
                                    param_grid = params,
                                    #n_iter=50,
                                    scoring='accuracy',
                                    verbose=3,
                                    cv=skf.split(X_train,y_train),
                                    return_train_score=True)

gsearch1.fit(X_train,y_train)

Fitting 2 folds for each of 1296 candidates, totalling 2592 fits
[CV] colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.4 
[CV]  colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.4, score=0.5048417635921018, total=   3.2s
[CV] colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.3s remaining:    0.0s


[CV]  colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.4, score=0.5020829951847644, total=   2.7s
[CV] colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.6 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.2s remaining:    0.0s


[CV]  colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.6, score=0.5059237219367054, total=   2.9s
[CV] colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.6, score=0.5012173348482389, total=   2.7s
[CV] colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.8 
[CV]  colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.8, score=0.5059237219367054, total=   2.5s
[CV] colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.8 
[CV]  colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weight=3, subsample=0.8, score=0.5006221933668776, total=   2.6s
[CV] colsample_bytree=0.6, gamma=0.25, learning_rate=0.01, max_depth=1, min_child_weigh

...

[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.05, max_depth=5, min_child_weight=7, subsample=0.4, score=0.6234918573824596, total=  12.0s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.05, max_depth=5, min_child_weight=7, subsample=0.6 
[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.05, max_depth=5, min_child_weight=7, subsample=0.6, score=0.6259670002704896, total=  12.8s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.05, max_depth=5, min_child_weight=7, subsample=0.6 
[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.05, max_depth=5, min_child_weight=7, subsample=0.6, score=0.6222474706487042, total=  12.7s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.05, max_depth=5, min_child_weight=7, subsample=0.8 
[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.05, max_depth=5, min_child_weight=7, subsample=0.8, score=0.6265079794427915, total=  12.0s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.05, max_depth=5, min_child_weigh

[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=1, min_child_weight=5, subsample=0.8, score=0.5881525561265891, total=   3.0s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=1, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=1, min_child_weight=5, subsample=0.8, score=0.5884326137531786, total=   3.0s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=1, min_child_weight=7, subsample=0.4 
[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=1, min_child_weight=7, subsample=0.4, score=0.592480389505004, total=   3.2s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=1, min_child_weight=7, subsample=0.4 
[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=1, min_child_weight=7, subsample=0.4, score=0.5928691229778715, total=   3.3s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=1, min_child_weight=7, subs

[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=5, min_child_weight=5, subsample=0.4, score=0.6167829897743873, total=  11.9s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=5, min_child_weight=5, subsample=0.6 
[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=5, min_child_weight=5, subsample=0.6, score=0.6268325669461726, total=  12.7s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=5, min_child_weight=5, subsample=0.6 
[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=5, min_child_weight=5, subsample=0.6, score=0.6207325650597847, total=  12.6s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=5, min_child_weight=5, subsample=0.8 
[CV]  colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=5, min_child_weight=5, subsample=0.8, score=0.6287800919664593, total=  11.7s
[CV] colsample_bytree=1.0, gamma=0.75, learning_rate=0.1, max_depth=5, min_child_weight=5, sub

[Parallel(n_jobs=1)]: Done 2592 out of 2592 | elapsed: 408.5min finished


GridSearchCV(cv=<generator object _BaseKFold.split at 0x1a11c9da98>,
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=300, nthread=-1,
       objective='multi:softmax', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'min_child_weight': [3, 5, 7], 'gamma': [0.25, 0.5, 0.75], 'subsample': [0.4, 0.6, 0.8], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [1, 3, 5, 7], 'learning_rate': [0.01, 0.025, 0.05, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=3)

In [36]:
# Best parameters
gsearch1.best_params_, gsearch1.best_score_

({'colsample_bytree': 0.8,
  'gamma': 0.75,
  'learning_rate': 0.05,
  'max_depth': 5,
  'min_child_weight': 3,
  'subsample': 0.4},
 0.62637957152131574)

In [ ]:
## If best param is largest or smallest tried, change range in grid search

In [32]:
params2 = {'n_estimators': [100, 200, 300, 400, 500]
        }

xgb2 = XGBClassifier(objective= 'multi:softmax',
              seed=27,
              #Add best params from gsearch1 here:
              subsample= 0.6,
              min_child_weight= 5,
              max_depth= 5,
              learning_rate= 0.05,
              gamma= 0.5,
              colsample_bytree= 0.8
             )

gsearch2 = GridSearchCV(estimator = xgb2, 
                        param_grid = params2, 
                        scoring='accuracy',
                        cv=skf.split(X_train,y_train))

In [33]:
gsearch2.fit(X_train,y_train)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x1a10429360>,
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.5, learning_rate=0.05, max_delta_step=0, max_depth=5,
       min_child_weight=5, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softmax', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.6),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200, 300, 400, 500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [34]:
gsearch2.best_params_, gsearch2.best_score_

({'n_estimators': 300}, 0.62478359662410732)

In [31]:
#gsearch2.best_params_, gsearch2.best_score_

({'n_estimators': 600}, 0.62426963860636231)